In [1]:
import sqlite3
import schedule
import time
import SQLite3_functions

In [2]:
Bdd = 'PiouPiou.db'
conn = sqlite3.connect(Bdd)

In [3]:
sql_insertion_table = ('''CREATE TABLE IF NOT EXISTS Infos(
                            id integer,
                            location_date DATE,
                            latitude REAL,
                            longitude REAL,
                            success TEXT,
                            measurements_date DATE,
                            pressure TEXT,
                            wind_heading REAL,
                            wind_speed_avg REAL,
                            wind_speed_max REAL,
                            wind_speed_min REAL,
                            name TEXT,
                            status_date DATE,
                            snr REAL,
                            state TEXT);
''')

In [4]:
SQLite3_functions.creer_table(conn, sql_insertion_table)

La table a été créée avec succès


In [5]:
import requests
import json

In [6]:
response = requests.get('http://api.pioupiou.fr/v1/live-with-meta/all')

In [7]:
response_1 = json.loads(response.text)

In [8]:
response_1.keys

<function dict.keys>

In [9]:
data = []
loc = [410,113,307]

for e in loc:
    k = '/'+str(e)
    response = requests.get('http://api.pioupiou.fr/v1/live-with-meta'+k+'')
    # Création dictionnaire Python méthode json.loads et décodeur .text
    response_info = json.loads(response.text)

    data.append([response_info['data']['id'], response_info['data']['location']['date'], response_info['data']['location']['latitude'], 
                    response_info['data']['location']['longitude'], response_info['data']['location']['success'], 
                    response_info['data']['measurements']['date'], response_info['data']['measurements']['pressure'], 
                    response_info['data']['measurements']['wind_heading'], response_info['data']['measurements']['wind_speed_avg'],  
                    response_info['data']['measurements']['wind_speed_max'], response_info['data']['measurements']['wind_speed_min'], 
                    response_info['data']['meta']['name'], response_info['data']['meta']['date'], response_info['data']['status']['snr'], 
                    response_info['data']['status']['state']]
                )
        # print(response_info)

In [10]:
Insérer_Données = ('''
    INSERT INTO Infos (id, location_date, latitude, longitude, success, measurements_date, pressure, 
    wind_heading, wind_speed_avg, wind_speed_max, wind_speed_min, name, status_date, snr, state)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    ''')

In [11]:
def boucle():
    SQLite3_functions.inserer_donnees(conn, Insérer_Données, data)

In [12]:
source = conn
bck = sqlite3.connect('backup.db')
source.backup(bck)

In [13]:
def check_table():
    
    with sqlite3.connect('PiouPiou.db') as db:
        
        cursor = db.cursor()
        cursor.execute('''SELECT COUNT(rowid) 
                        FROM Infos''')
        result = cursor.fetchall()
        return result[0][0]

In [14]:
check_table()

10

In [15]:
def del_rows():

    with sqlite3.connect('PiouPiou.db') as db:
        
        cursor = db.cursor()
        
        cursor.execute('''DELETE
                          FROM Infos
                          WHERE rowid IN (Select rowid from Infos limit 4);
                        ''')

In [16]:
def gestion():

    from datetime import datetime
    
    if check_table() > 9 :
        del_rows()
        print ('Delete succeed :', check_table(), 'rows in Database', datetime.now())

    else :
        print ('Nothing to delete', datetime.now())

In [17]:
def backup(conn):
    
    bck = sqlite3.connect('PiouPiou.db')
    
    with bck :
        conn.backup(bck)
        print ('Backup succeed', datetime.now())
    
    bck.close()
    conn.close()

In [19]:
def backup_table():
    
    if check_table() == 10 :
        backup(conn)

In [20]:
def foil():
    
    gestion()
    boucle()      
    
    schedule.every(1).minutes.do(boucle)
    schedule.every(13).seconds.do(backup_table)
    schedule.every(50).seconds.do(gestion)
    
    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
foil()

Delete succeed : 6 rows in Database 2021-03-17 13:57:07.740611
Les données ont été insérées avec succès
Nothing to delete 2021-03-17 13:57:58.316903
Les données ont été insérées avec succès
Delete succeed : 8 rows in Database 2021-03-17 13:58:48.909391
Les données ont été insérées avec succès
Delete succeed : 7 rows in Database 2021-03-17 13:59:39.550536
Les données ont été insérées avec succès
